# Final Project Notebook

### NOTE: If you build and run a notebook in the cloud, just copy it down in place of this one!  
#### Be sure to have all your output captured within the notebook!
#### <span style="background:yellow">Be sure to save your work early and often!</span>

![Specific_Project_1.png MISSING](../images/Specific_Project_1.png)

In [ ]:
#run instance
gcloud beta compute --project=cloud15 instances create dsa-mini-project-lcn1055-mod8 --zone=us-central1-a --machine-type=f1-micro --subnet=default --network-tier=PREMIUM --no-restart-on-failure --maintenance-policy=TERMINATE --preemptible --service-account=1014104180905-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --image=debian-10-buster-v20210609 --image-project=debian-cloud --boot-disk-size=10GB --boot-disk-type=pd-balanced --boot-disk-device-name=dsa-mini-project-lcn1055-1 --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=module=6,user=lcn1055 --reservation-affinity=any
# I stop and allow api to access before start again and ssh in.

In [ ]:
sudo apt install python3-venv python3-pip
sudo apt install wget
wget https://bootstrap.pypa.io/get-pip.py
sudo python3 get-pip.py
#create virtual env to install need libraries
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade setuptools
pip3 install --upgrade google-api-python-client
pip3 install --upgrade google-cloud-storage
pip3 install --upgrade feedparser
pip3 install --upgrade beautifulsoup4
pip3 install --upgrade datetime

pip3 install --no-cache-dir --force-reinstall grpcio
#pip3 install google-cloud-storage #pip3 install --upgrade google-cloud-datastore

In [ ]:
#Next, we need to run gcloud to authorize API from the instance's terminal
gcloud auth application-default login



# Add code as needed in the cells below to produce your analytical products

In [ ]:
## Include whichever API are appropriate for your cloud provider

####################################
# File: redditfeed.py
####################################
import feedparser
from bs4 import BeautifulSoup
from bs4.element import Comment

from datetime import datetime
import json

# for bucket
import os
from google.cloud import storage

# for noSQL Firestore Datastore
import googleapiclient.discovery
from google.cloud import datastore
import requests


##Write ReadReddit To Retriev Dictionary DataSET

# Functions from: https://stackoverflow.com/questions/1936466/beautifulsoup-grab-visible-webpage-text

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

# Define URL of the RSS Feed I want
def readRedditDict():
    a_reddit_rss_url = 'http://www.reddit.com/new/.rss?sort=new'

    feed = feedparser.parse( a_reddit_rss_url )
    results = {}
    seq =0
    if (feed['bozo'] == 1):
        print("Error Reading/Parsing Feed XML Data")    
    else:
        for item in feed[ "items" ]:
            dttm = item[ "date" ] #"2021-06-17T18:50:49+00:00_2"
            seq +=1
            seqdttm = dttm[:4]+dttm[5:7]+dttm[8:10]+dttm[11:13]+dttm[14:16]+dttm[17:19]+"_"+str(seq)
            results[seqdttm]= \
                    {
                    "posteddatetime":item[ "date" ] ,
                    "title":item[ "title" ],
                    "summary":text_from_html(item[ "summary" ]),
                    "link":item[ "link" ]
                    }

    return results     

############################################
##Write ReadReddit To Retriev Dictionary DataSET

PROJECT='cloud15'

#vbuckname = 'dsa_mini_project_lcn1055'
#filename = 'RedditJson_'+datetime.now().strftime("%Y%m%d%H%M")  #length=19



def list_blobs(bucket_name, blobnamepattern):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs()
    retblobs = []
    for blob in blobs:
        if blobnamepattern == '':
            print(blob.name)
            retblobs.append(blob.name)
        else:
            if blobnamepattern in blob.name:
                retblobs.append(blob.name)
    return retblobs    
        

def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.
    This can be used to list all blobs in a "folder", e.g. "public/".
    The delimiter argument can be used to restrict the results to only the
    "files" in the given "folder". Without the delimiter, the entire tree under
    the prefix is returned. For example, given these blobs:
        /a/1.txt
        /a/b/2.txt
    If you just specify prefix = '/a', you'll get back:
        /a/1.txt
        /a/b/2.txt
    However, if you specify prefix='/a' and delimiter='/', you'll get back:
        /a/1.txt
    """
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix=prefix, delimiter=delimiter)

    print('Blobs:')
    for blob in blobs:
        print(blob.name)

    if delimiter:
        print('Prefixes:')
        for prefix in blobs.prefixes:
            print(prefix)
            


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))


def upload_as_blob(bucket_name, source_data, destination_blob_name, content_type='text/plain'):
    """Uploads a file to the bucket."""
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    """blob.upload_from_filename(source_file_name)"""
    blob.upload_from_string(source_data, content_type=content_type)
    
    print('Data uploaded to {}.'.format(destination_blob_name))


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

    print('Blob {} deleted.'.format(blob_name))



def read_blob_dict(bucket_name, source_blob_name):
    """Read a blob from the bucket into Text"""
    
    storage_client = storage.Client(project=PROJECT)
    bucket = storage_client.get_bucket(bucket_name)

    # download as string
    json_data = bucket.get_blob(source_blob_name).download_as_string()
    json_dict = json.loads(json_data.decode('utf-8'))   

    return json_dict
    """
    #blob = bucket.get_blob(source_blob_name)
    with open(json_data) as f:
        data = json.load(json_data)
       itSourceId'] = data['RedditSourceId']
    """
## Write your additional Code Segments here, 
##  because your VM will get deleted


vbuckname = 'dsa_mini_project_lcn1055'
filename = 'RedditJson_' + datetime.now().strftime("%Y%m%d%H%M")  #length=19
list_blobs(vbuckname,'')
print('Before load ---------')
# upload_as_blob(vbuckname, json.dumps(readRedditDict(), indent = 4), filename)
list_blobs(vbuckname,'')
print('After load ---------')







In [ ]:
######################################
# File: bucket2NoSQL.py
#######################
import os
from google.cloud import storage
import json
# for noSQL Firestore Datastore
import googleapiclient.discovery
from google.cloud import datastore
import requests

from redditfeed import list_blobs, read_blob_dict
######################################

#projectid=cloud15

def create_client(project_id):
    return datastore.Client(project_id)

def add_redditItem(projectid, redditsourceid, redditsourcedtt, title, summary, link):
    client=create_client(projectid)
    key = client.key('Reddit')
    print('Entity is Reddit')
    reddit = datastore.Entity(
        key, exclude_from_indexes=['Title','Summary','Link'])

    reddit.update({
        'RedditSourceId': redditsourceid, #datetime.datetime.utcnow(),
        'RedditSourceDtt': redditsourcedtt,
        'Title': title,
        'Summary':summary,
        'Link': link
    })

    client.put(reddit)

    return reddit.key


def mark_done(client, reddit_id):
    with client.transaction():
        key = client.key('Reddit', reddit_id)
        task = client.get(key)

        if not task:
            raise ValueError(
                'Reddit {} does not exist.'.format(reddit_id))

        reddit['Title'] = 'Done'

        client.put(reddit)

        
def delete_task(client, reddit_id):
    key = client.key('Reddit', reddit_id)
    client.delete(key)  
    

def list_tasks(client):
    query = client.query(kind='Reddit')
    query.order = ['RedditSourceId']

    return list(query.fetch())    


### Loop through bucket to look for a file RedditJsonXXXX to  store data into NoSQL
vbuckname = 'dsa_mini_project_lcn1055'
fnamepattern = 'RedditJson_' 
PROJECT='cloud15'

# Listing RedditJson blob name in the bucket in order to put into NoSQL
redditjson = list_blobs(vbuckname, fnamepattern)
#client = storage.Client(project=PROJECT)
#each  bucketname
for bname in redditjson:
    # Grab data each blb and read row by row
    datadict  = read_blob_dict(vbuckname, bname)
    stop =0
    print('File No.'+ str(stop))
    for k,v in datadict.items():
        stop+=1
    # start write into  firestore-datasore ###
        # print(k)
        c1=str(k)
        
        c2=''
        c3=''
        c4=''
        c5=''
        # v="{'posteddatetime': '2021-06-19T03:07:43+00:00', 'title': 'ASUS TUF Dash 15, 144Hz FHD, RTX 3050 Ti, i7-11370H, 8GB 512GB, $949 - ASUS ROG 13.4" AMD Ryzen 9, RTX 3050 Ti, 16GB 1TB, $1499 - ASUS TUF Gaming F15, 144Hz FHD, i7-11800H, RTX 3050 Ti, 16GB 512GB, $1199 - all available for order', 'summary': '     submitted by /u/gamersecret2 to r/GamingLaptops   [link]  [comments] ', 'link': 'https://www.reddit.com/r/GamingLaptops/comments/o36rs7/asus_tuf_dash_15_144hz_fhd_rtx_3050_ti_i711370h/'}"
        cntiteminOneFile=0
        for kk,vv in v.items():
           cntiteminOneFile+=1  
           if kk=='RedditSourceId' :
               print('****'+v)
               c1=vv
           elif kk=='posteddatetime' or kk=='datetime':
               c2=vv
           elif kk=='title':
               c3=vv
           elif kk=='summary':
               c4=vv
           elif kk=='link':
               c5=vv
           else:
               print("CANNOT FIND:" + kk)
           
        print(c1+'\n'+c2+'\n'+c3+'\n'+c4+'\n'+c5) 
        add_redditItem(PROJECT,c1,c2,c3,c4,c5)
   
        #print(cntiteminOneFile)
    print('****************************\n')
    #print('\n'.join(json.dumps(item) for item in json_list))
    
    # add_redditItem(client, redditsourceid, redditsourcedtt, title, summary, link):


In [ ]:
######################################
# File: drawSummary.py #

# Obj: To write the outut from datastore into csv
#######################
                                               
from google.cloud import datastore
import csv
# For help authenticating your client, visit
# https://cloud.google.com/docs/authentication/getting-started
client = datastore.Client()
query = client.query(kind="Reddit")
query.add_filter("RedditSourceId", ">", "20210615")
# query.order = ["Title"]
result = list(query.fetch())
with open('./output.csv','w') as f:
    writer =  csv.writer(f)
    writer.writerow(['RedditDtt','Title','Link','Summary'])
    for i in result:
        c1=i['RedditSourceDtt'] 
        c2=i['Title']
        c3=i['Summary']     
        c4=i['Link']
        data=[c1,c2,c3,c4]
        writer.writerow(data) 


## Preparing your submission

### Deliverables: 
   1. This or a replacement Notebook
   1. An aggregateion of data in tabular format that conveyes something interesting about the Reddit RSS feed during your scraping.
     * The table can be embedded or uploaded into this folder (CSV or Excel)
   1. One or more data visualizations

Imbed your image into this page by saving your data visualization as: `FINAL_PROJECT_IMAGE.png`  
Upload it to the `module8/exercises/` folder.

If you need to, change the file type to `.jpg` or `.jpeg` or ... whatever, then update the link in this cell (double click to edit).  
Then re-run this markdown cell to see it.

![FINAL_PROJECT_IMAGE.png MISSING](./exercises/FINAL_PROJECT_IMAGE.png)

---
## Summarize in the fields below
 1. Describe the overall process and components you used for the project.
 2. What is the key insight from the tabularization?
 3. What is the key insight from the visualization?

In [ ]:
### 3.
# --------------------------

#Next, we need to run gcloud to authorize API from the instance's terminal
gcloud auth application-default login






### 4
# --------------------------

# Set up Google Firestore DataStore and create an entity with 5 columns
#Entity: Reddit
#Column: RedditSourceId, RedditSourceDatetime, Title, Summary, Link
    
![Reddit NoSQL Result](../images/Mod8_DataStore_RedditEntity.png)

### 5
# --------------------------

# Set up Google Cloudstore Bucket
Name: dsa_mini_project_lcn1055
# Set up API to allow access

![Show Reddit Json Bucket](../images/Mod8_GCPBucket_storingRedditJsonFormat.png)
    

### 6
# --------------------------

#Next, we need to run gcloud to authorize API from the instance's terminal
# In the Instance SSH, I have run bucket2NoSQL.py to read reddit and put into json into the bucket and read to store into NoSQL

Filename is redditfeed.py and bucket2NoSQL.py
can be found under Exercises Folder

### 7.
# --------------------------

# Next, We run script to generate Visualization - output.csv from DataScore using 
# the script called drawSummary.py
# Here is the result

![Output From DataStore Pull](../images/output_csv_screenshot.png)


In [ ]:
What is the key insight from the tabularization?
-> Depending on RedditSourceDateTime
What is the key insight from the visualization?
-> There are 175 records and we can pull based on the column we have set for Indices.


# Save your Notebook!